# Домашнее задание # 2

# Тензорные и Матричные Разложения

В данном ноутбуке вам предстоит применять матричные и тензорные разложения для оптимизации модели по количеству хранимых параметров.
Для начала мы познакомимся с тем как устроен BERT и применим его на задаче SST2 для оценки сентимента.
Далее предстоит применять SVD к слоям берта с целью их сжатия, и в конечном итоге попробовать тензорные разложения

# Задание 1 (25 Баллов всего)
В данном задании вам предлагается использовать SVD, чтобы сэкономить параметров у модели во время инференса

Задание делится на несколько частей
## 1. Реализация (10 Баллов)
Напишите реализацию применения SVD к некоторым слоям и некоторым матрицам проекции на выбор. Эта реализация может быть разной, вы можете добавить функцию внутрь класса БЕРТа и внутрь его проекций, которая добавляет им функциональности. Либо это может быть функция которая итерируется по слоям и подменяет их на новый кастомный слой, который поддерживает SVD разложение и оптимальный инференс с его помощью (нужный порядок умножения матриц на активации).
Эта реализация должна поддерживать возможность выбирать какие слои сжимать и какие матрицы в них

## 2. Чувствительность (6 Баллов)
Примените SVD к некоторым слоям и некоторым матрицам. Обратите внимание, что они по разному будут чувствительны к разложению (вспомните график из статьи LASER с лекции). Протестируйте какие слои и какие матрицы более или менее чувствительны к разложению. Сделайте выводы что лучше сжимать, а что лучше не трогать и насколько сильно можно сжимать.

## 3. Дообучение (7 Баллов)
Выберите несколько лучших постановок из прошлого пункта, эти постановки должны отличаться по количеству параметров и степени сжатия. Зафиксировав эти слои и эти матрицы, далее дообучите их на тренировочной выборке SST2. Рекомендуется пользоваться приведенными выше функциями, они позволят вам сосредоточиться на разложении, а не на рутинных функциях обучения.

По итогу предлагается исследовать как просело или выросло качество по сравнению с изначальным, а также как оно зависит от степени сжатия и количества параметров.

## 4. Выводы (2 Балла)
Сделайте выводы на основе полученных в этом задании результатов, попробуйте понять почему так получилось. Будет интересно и здорово, если Ваши выводы совпадут с какой-нибудь статьей и вы сошлетесь на нее.



In [2]:
!pip install -q evaluate accelerate tensorly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [78]:
import math
import os
import warnings
from dataclasses import dataclass
from typing import List, Optional, Tuple, Union
import numpy as np
import torch
import torch.utils.checkpoint
from packaging import version
from torch import nn, Tensor
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers import AutoConfig, BertForSequenceClassification, BertTokenizer,Trainer, TrainingArguments, DataCollatorWithPadding
import evaluate
from datasets import load_dataset
from tqdm import tqdm



# Bert

Первым делом обозначим активацию, которая будет у нас почти везде. В случае BERT это у нас GELU, в описании к функции небольшая справка и ссылка на статью

In [4]:
class GELUActivation(nn.Module):
    """
    Original Implementation of the GELU activation function in Google BERT repo when initially created. For
    information: OpenAI GPT's GELU is slightly different (and gives slightly different results): 0.5 * x * (1 +
    torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3)))) This is now written in C in nn.functional
    Also see the Gaussian Error Linear Units paper: https://arxiv.org/abs/1606.08415
    """

    def __init__(self, use_gelu_python: bool = False):
        super().__init__()
        if use_gelu_python:
            self.act = self._gelu_python
        else:
            self.act = nn.functional.gelu

    def _gelu_python(self, input: Tensor) -> Tensor:
        return input * 0.5 * (1.0 + torch.erf(input / math.sqrt(2.0)))

    def forward(self, input: Tensor) -> Tensor:
        return self.act(input)

Следом нам предстоит регулярно делать эмбеддинг к нашей модели. У Берта 3 эмбеддинга
1. для токена. Он нам нужен чтобы правильно кодировать токены
2. для позиции. Он нам нужен чтобы кодировать позицию. Позиционные токены иногда делают синусом и косинусом, но часто эффективнее просто их учить.
3. для типа токена. Такой вид токена был нужен в оригинале для задачи NSP, нам он не будет нужен



In [5]:
class BertEmbeddings(nn.Module):
    """Construct the embeddings from word, position and token_type embeddings."""

    def __init__(self, config):
        super().__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)

        # self.LayerNorm is not snake-cased to stick with TensorFlow model variable name and be able to load
        # any TensorFlow checkpoint file
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        # position_ids (1, len position emb) is contiguous in memory and exported when serialized
        self.position_embedding_type = getattr(config, "position_embedding_type", "absolute")
        self.register_buffer(
            "position_ids", torch.arange(config.max_position_embeddings).expand((1, -1)), persistent=False
        )
        self.register_buffer(
            "token_type_ids", torch.zeros(self.position_ids.size(), dtype=torch.long), persistent=False
        )

    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        past_key_values_length: int = 0,
    ) -> torch.Tensor:
        if input_ids is not None:
            input_shape = input_ids.size()
        else:
            input_shape = inputs_embeds.size()[:-1]

        seq_length = input_shape[1]

        if position_ids is None:
            position_ids = self.position_ids[:, past_key_values_length : seq_length + past_key_values_length]

        # Setting the token_type_ids to the registered buffer in constructor where it is all zeros, which usually occurs
        # when its auto-generated, registered buffer helps users when tracing the model without passing token_type_ids, solves
        # issue #5664
        if token_type_ids is None:
            if hasattr(self, "token_type_ids"):
                buffered_token_type_ids = self.token_type_ids[:, :seq_length]
                buffered_token_type_ids_expanded = buffered_token_type_ids.expand(input_shape[0], seq_length)
                token_type_ids = buffered_token_type_ids_expanded
            else:
                token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=self.position_ids.device)

        if inputs_embeds is None:
            inputs_embeds = self.word_embeddings(input_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)

        embeddings = inputs_embeds + token_type_embeddings
        if self.position_embedding_type == "absolute":
            position_embeddings = self.position_embeddings(position_ids)
            embeddings += position_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

Следом нам следует сделать механизм внимания. У нас будет несколько голов, проекция для ключей, запросов и значений в виде линейных слоев. Далее потребуется еще одна выходная проекция, она будет в отдельном классе с нормализацией и дропаутом.

В реализации может встречаться условие на то является ли текущий слой частью декдора. В нашем случае не является, нам потребуется только энкодер, эти части можно игнорировать.

In [6]:
class BertSelfAttention(nn.Module):
    def __init__(self, config, position_embedding_type=None):
        super().__init__()
        if config.hidden_size % config.num_attention_heads != 0 and not hasattr(config, "embedding_size"):
            raise ValueError(
                f"The hidden size ({config.hidden_size}) is not a multiple of the number of attention "
                f"heads ({config.num_attention_heads})"
            )

        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)
        self.position_embedding_type = position_embedding_type or getattr(
            config, "position_embedding_type", "absolute"
        )
        if self.position_embedding_type == "relative_key" or self.position_embedding_type == "relative_key_query":
            self.max_position_embeddings = config.max_position_embeddings
            self.distance_embedding = nn.Embedding(2 * config.max_position_embeddings - 1, self.attention_head_size)

        self.is_decoder = config.is_decoder

    def transpose_for_scores(self, x: torch.Tensor) -> torch.Tensor:
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]:
        mixed_query_layer = self.query(hidden_states)

        # If this is instantiated as a cross-attention module, the keys
        # and values come from an encoder; the attention mask needs to be
        # such that the encoder's padding tokens are not attended to.
        is_cross_attention = encoder_hidden_states is not None

        if is_cross_attention and past_key_value is not None:
            # reuse k,v, cross_attentions
            key_layer = past_key_value[0]
            value_layer = past_key_value[1]
            attention_mask = encoder_attention_mask
        elif is_cross_attention:
            key_layer = self.transpose_for_scores(self.key(encoder_hidden_states))
            value_layer = self.transpose_for_scores(self.value(encoder_hidden_states))
            attention_mask = encoder_attention_mask
        elif past_key_value is not None:
            key_layer = self.transpose_for_scores(self.key(hidden_states))
            value_layer = self.transpose_for_scores(self.value(hidden_states))
            key_layer = torch.cat([past_key_value[0], key_layer], dim=2)
            value_layer = torch.cat([past_key_value[1], value_layer], dim=2)
        else:
            key_layer = self.transpose_for_scores(self.key(hidden_states))
            value_layer = self.transpose_for_scores(self.value(hidden_states))

        query_layer = self.transpose_for_scores(mixed_query_layer)

        use_cache = past_key_value is not None
        if self.is_decoder:
            # if cross_attention save Tuple(torch.Tensor, torch.Tensor) of all cross attention key/value_states.
            # Further calls to cross_attention layer can then reuse all cross-attention
            # key/value_states (first "if" case)
            # if uni-directional self-attention (decoder) save Tuple(torch.Tensor, torch.Tensor) of
            # all previous decoder key/value_states. Further calls to uni-directional self-attention
            # can concat previous decoder key/value_states to current projected key/value_states (third "elif" case)
            # if encoder bi-directional self-attention `past_key_value` is always `None`
            past_key_value = (key_layer, value_layer)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))

        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        if attention_mask is not None:
            # Apply the attention mask is (precomputed for all layers in BertModel forward() function)
            attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = nn.functional.softmax(attention_scores, dim=-1)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        # Mask heads if we want to
        if head_mask is not None:
            attention_probs = attention_probs * head_mask

        context_layer = torch.matmul(attention_probs, value_layer)

        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(new_context_layer_shape)

        outputs = (context_layer, attention_probs) if output_attentions else (context_layer,)

        if self.is_decoder:
            outputs = outputs + (past_key_value,)
        return outputs


In [7]:
class BertSelfOutput(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states: torch.Tensor, input_tensor: torch.Tensor) -> torch.Tensor:
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states

class BertAttention(nn.Module):
    def __init__(self, config, position_embedding_type=None):
        super().__init__()
        self.self = BertSelfAttention(
            config, position_embedding_type=position_embedding_type
        )
        self.output = BertSelfOutput(config)

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]:
        self_outputs = self.self(
            hidden_states,
            attention_mask,
            head_mask,
            encoder_hidden_states,
            encoder_attention_mask,
            past_key_value,
            output_attentions,
        )
        attention_output = self.output(self_outputs[0], hidden_states)
        outputs = (attention_output,) + self_outputs[1:]  # add attentions if we output them
        return outputs

Ну и конечно нам нужен Feed Forward слой, для него у нас будет проекция в более высокую размерность а потом обратно в размерность нашей модели

In [8]:
class BertIntermediate(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.intermediate_size)
        self.intermediate_act_fn = GELUActivation()

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        hidden_states = self.dense(hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states)
        return hidden_states


class BertOutput(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.intermediate_size, config.hidden_size)
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states: torch.Tensor, input_tensor: torch.Tensor) -> torch.Tensor:
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states


Все вышеперечисленные слои мы собираем в один слой берта. Еще раз отметим, что есть часть имплементации под бертовый слой на декодер, не обращайте на него внимания, он нам не потребуется

In [9]:
class BertLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.seq_len_dim = 1
        self.attention = BertAttention(config)
        self.is_decoder = config.is_decoder
        self.add_cross_attention = config.add_cross_attention
        if self.add_cross_attention:
            if not self.is_decoder:
                raise ValueError(f"{self} should be used as a decoder model if cross attention is added")
            self.crossattention = BertAttention(config, position_embedding_type="absolute")
        self.intermediate = BertIntermediate(config)
        self.output = BertOutput(config)

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]:
        # decoder uni-directional self-attention cached key/values tuple is at positions 1,2
        self_attn_past_key_value = past_key_value[:2] if past_key_value is not None else None
        self_attention_outputs = self.attention(
            hidden_states,
            attention_mask,
            head_mask,
            output_attentions=output_attentions,
            past_key_value=self_attn_past_key_value,
        )
        attention_output = self_attention_outputs[0]

        # if decoder, the last output is tuple of self-attn cache
        if self.is_decoder:
            outputs = self_attention_outputs[1:-1]
            present_key_value = self_attention_outputs[-1]
        else:
            outputs = self_attention_outputs[1:]  # add self attentions if we output attention weights

        cross_attn_present_key_value = None
        if self.is_decoder and encoder_hidden_states is not None:
            if not hasattr(self, "crossattention"):
                raise ValueError(
                    f"If `encoder_hidden_states` are passed, {self} has to be instantiated with cross-attention layers"
                    " by setting `config.add_cross_attention=True`"
                )

            # cross_attn cached key/values tuple is at positions 3,4 of past_key_value tuple
            cross_attn_past_key_value = past_key_value[-2:] if past_key_value is not None else None
            cross_attention_outputs = self.crossattention(
                attention_output,
                attention_mask,
                head_mask,
                encoder_hidden_states,
                encoder_attention_mask,
                cross_attn_past_key_value,
                output_attentions,
            )
            attention_output = cross_attention_outputs[0]
            outputs = outputs + cross_attention_outputs[1:-1]  # add cross attentions if we output attention weights

            # add cross-attn cache to positions 3,4 of present_key_value tuple
            cross_attn_present_key_value = cross_attention_outputs[-1]
            present_key_value = present_key_value + cross_attn_present_key_value

        layer_output = self.feed_forward_chunk(attention_output)
        outputs = (layer_output,) + outputs

        # if decoder, return the attn key/values as the last output
        if self.is_decoder:
            outputs = outputs + (present_key_value,)

        return outputs

    def feed_forward_chunk(self, attention_output):
        intermediate_output = self.intermediate(attention_output)
        layer_output = self.output(intermediate_output, attention_output)
        return layer_output


Далее мы делаем полный энкодер, который будет составлен из нескольких слоев берта. И также пуллер, сверху на 0 токене будет голова в виде линейного слоя.

In [10]:
class BertEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.layer = nn.ModuleList([BertLayer(config) for _ in range(config.num_hidden_layers)])

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = False,
        output_hidden_states: Optional[bool] = False,
        return_dict: Optional[bool] = True,
    ) -> Tuple[torch.Tensor]:
        all_hidden_states = () if output_hidden_states else None
        all_self_attentions = () if output_attentions else None
        all_cross_attentions = () if output_attentions and self.config.add_cross_attention else None

        next_decoder_cache = () if use_cache else None
        for i, layer_module in enumerate(self.layer):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            layer_head_mask = head_mask[i] if head_mask is not None else None
            past_key_value = past_key_values[i] if past_key_values is not None else None

            layer_outputs = layer_module(
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                    past_key_value,
                    output_attentions,
            )

            hidden_states = layer_outputs[0]
            if use_cache:
                next_decoder_cache += (layer_outputs[-1],)
            if output_attentions:
                all_self_attentions = all_self_attentions + (layer_outputs[1],)
                if self.config.add_cross_attention:
                    all_cross_attentions = all_cross_attentions + (layer_outputs[2],)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        return tuple(
                v
                for v in [
                    hidden_states,
                    next_decoder_cache,
                    all_hidden_states,
                    all_self_attentions,
                    all_cross_attentions,
                ]
                if v is not None
        )


class BertPooler(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output


Ну и наконец полная модель, она соберет вместе эмбеддинги, энкодер и пуллер.

In [11]:
class BertModel(nn.Module):
    """

    The model can behave as an encoder (with only self-attention) as well as a decoder, in which case a layer of
    cross-attention is added between the self-attention layers, following the architecture described in [Attention is
    all you need](https://arxiv.org/abs/1706.03762) by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit,
    Llion Jones, Aidan N. Gomez, Lukasz Kaiser and Illia Polosukhin.

    To behave as an decoder the model needs to be initialized with the `is_decoder` argument of the configuration set
    to `True`. To be used in a Seq2Seq model, the model needs to initialized with both `is_decoder` argument and
    `add_cross_attention` set to `True`; an `encoder_hidden_states` is then expected as an input to the forward pass.
    """

    _no_split_modules = ["BertEmbeddings"]

    def __init__(self, config, add_pooling_layer=True):
        super().__init__()
        self.config = config

        self.embeddings = BertEmbeddings(config)
        self.encoder = BertEncoder(config)

        self.pooler = BertPooler(config) if add_pooling_layer else None

        self.position_embedding_type = config.position_embedding_type

    def get_input_embeddings(self):
        return self.embeddings.word_embeddings

    def set_input_embeddings(self, value):
        self.embeddings.word_embeddings = value

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        encoder_hidden_states: Optional[torch.Tensor] = None,
        encoder_attention_mask: Optional[torch.Tensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
    ) -> Tuple[torch.Tensor]:
        r"""
        encoder_hidden_states  (`torch.FloatTensor` of shape `(batch_size, sequence_length, hidden_size)`, *optional*):
            Sequence of hidden-states at the output of the last layer of the encoder. Used in the cross-attention if
            the model is configured as a decoder.
        encoder_attention_mask (`torch.FloatTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Mask to avoid performing attention on the padding token indices of the encoder input. This mask is used in
            the cross-attention if the model is configured as a decoder. Mask values selected in `[0, 1]`:

            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.
        past_key_values (`tuple(tuple(torch.FloatTensor))` of length `config.n_layers` with each tuple having 4 tensors of shape `(batch_size, num_heads, sequence_length - 1, embed_size_per_head)`):
            Contains precomputed key and value hidden states of the attention blocks. Can be used to speed up decoding.

            If `past_key_values` are used, the user can optionally input only the last `decoder_input_ids` (those that
            don't have their past key value states given to this model) of shape `(batch_size, 1)` instead of all
            `decoder_input_ids` of shape `(batch_size, sequence_length)`.
        use_cache (`bool`, *optional*):
            If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding (see
            `past_key_values`).
        """
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )

        if self.config.is_decoder:
            use_cache = use_cache if use_cache is not None else self.config.use_cache
        else:
            use_cache = False

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            input_shape = input_ids.size()
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        batch_size, seq_length = input_shape
        device = input_ids.device if input_ids is not None else inputs_embeds.device

        # past_key_values_length
        past_key_values_length = past_key_values[0][0].shape[2] if past_key_values is not None else 0

        if token_type_ids is None:
            if hasattr(self.embeddings, "token_type_ids"):
                buffered_token_type_ids = self.embeddings.token_type_ids[:, :seq_length]
                buffered_token_type_ids_expanded = buffered_token_type_ids.expand(batch_size, seq_length)
                token_type_ids = buffered_token_type_ids_expanded
            else:
                token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)

        embedding_output = self.embeddings(
            input_ids=input_ids,
            position_ids=position_ids,
            token_type_ids=token_type_ids,
            inputs_embeds=inputs_embeds,
            past_key_values_length=past_key_values_length,
        )

        if attention_mask is None:
            attention_mask = torch.ones((batch_size, seq_length + past_key_values_length), device=device)

        extended_attention_mask = self.get_extended_attention_mask(attention_mask, input_shape)


        encoder_extended_attention_mask = None

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
        # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
        head_mask = [None] * self.config.num_hidden_layers #self.get_head_mask(head_mask, self.config.num_hidden_layers)

        encoder_outputs = self.encoder(
            embedding_output,
            attention_mask=extended_attention_mask,
            head_mask=head_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_extended_attention_mask,
            past_key_values=past_key_values,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )
        sequence_output = encoder_outputs[0]
        pooled_output = self.pooler(sequence_output) if self.pooler is not None else None

        return (sequence_output, pooled_output) + encoder_outputs[1:]


    def get_extended_attention_mask(
        self, attention_mask: Tensor, input_shape: Tuple[int], device: torch.device = None, dtype: torch.float = None
    ) -> Tensor:
        """
        Makes broadcastable attention and causal masks so that future and masked tokens are ignored.

        Arguments:
            attention_mask (`torch.Tensor`):
                Mask with ones indicating tokens to attend to, zeros for tokens to ignore.
            input_shape (`Tuple[int]`):
                The shape of the input to the model.

        Returns:
            `torch.Tensor` The extended attention mask, with a the same dtype as `attention_mask.dtype`.
        """
        if dtype is None:
            dtype = torch.float

        # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
        # ourselves in which case we just need to make it broadcastable to all heads.
        if attention_mask.dim() == 3:
            extended_attention_mask = attention_mask[:, None, :, :]
        elif attention_mask.dim() == 2:
            # Provided a padding mask of dimensions [batch_size, seq_length]
            # - if the model is a decoder, apply a causal mask in addition to the padding mask
            # - if the model is an encoder, make the mask broadcastable to [batch_size, num_heads, seq_length, seq_length]
            extended_attention_mask = attention_mask[:, None, None, :]
        else:
            raise ValueError(
                f"Wrong shape for input_ids (shape {input_shape}) or attention_mask (shape {attention_mask.shape})"
            )

        # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
        # masked positions, this operation will create a tensor which is 0.0 for
        # positions we want to attend and the dtype's smallest value for masked positions.
        # Since we are adding it to the raw scores before the softmax, this is
        # effectively the same as removing these entirely.
        extended_attention_mask = extended_attention_mask.to(dtype=dtype)  # fp16 compatibility
        extended_attention_mask = (1.0 - extended_attention_mask) * torch.finfo(dtype).min
        return extended_attention_mask


Подобно реализации из HF у нас будет класс именно под классификацию, который использует основную модель и применяет к ней линейный слой. Также на форварде есть возможность сразу посчитать лосс если предоставить лейблы в качестве аргумента.

In [12]:
class BertForSequenceClassification(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_labels = config.num_labels
        self.config = config

        self.bert = BertModel(config)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        # Initialize weights and apply final processing
       # self.post_init()
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
    ) -> Tuple[torch.Tensor]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        output = (logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output

Мы возьмем модель уже обученную под SST2, маленького размера (чтобы было приятнее вычислять). И подгрузим веса в нашу реализацию. Веса доступны по ссылке
https://drive.google.com/file/d/1N4RGGFYmw7hVy6TR8SOyu82QmLbB20qy/view?usp=sharing

Нам здесь потребовалась собственная реализация для того, чтобы мы могли ее модифицировать далее

# Train

In [86]:
# ! gdown -O bert_tiny_sst.pt https://drive.google.com/file/d/1N4RGGFYmw7hVy6TR8SOyu82QmLbB20qy

In [87]:
config = AutoConfig.from_pretrained('VityaVitalich/bert-tiny-sst2')

bert = BertForSequenceClassification(config=config)

state_dict = torch.load('bert_tiny_sst.pt')
bert.load_state_dict(state_dict)

<ipython-input-87-117ea9f66065>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('bert_tiny_sst.pt')


<All keys matched successfully>

Ниже пример кода, который подгружает токенайзер, мы будем пользоваться стандартным и изменять его не будем.

Далее мы создаем трейн и тест сет, которые пойдут в HF Trainer на этапе обучения и тестирования

In [88]:
tokenizer = BertTokenizer.from_pretrained('VityaVitalich/bert-tiny-sst2')

def create_data(tokenizer):
  train_set = load_dataset('sst2', split='train').remove_columns(['idx'])
  val_set = load_dataset('sst2', split='validation').remove_columns(['idx'])

  def tokenize_func(examples):
      return tokenizer(examples["sentence"], max_length=256, padding='longest', truncation=True)

  encoded_dataset_train = train_set.map(tokenize_func, batched=True)
  encoded_dataset_test = val_set.map(tokenize_func, batched=True)
  data_collator = DataCollatorWithPadding(tokenizer)
  return encoded_dataset_train, encoded_dataset_test, data_collator

encoded_dataset_train, encoded_dataset_test, data_collator = create_data(tokenizer)


А также подгрузим метрику из библиотеки datasets. Это обычный Accuracy, мы могли бы его и сами сделать по итогу, но зачем изобретать велосипед. Такая функция передается в HF Trainer и вызывается каждый раз, когда мы хотим посчитать метрику

In [89]:
metric = evaluate.load('accuracy')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)


Объявим аргументы для тренировки и сам Trainer. Обращаю внимание, что аргументов у него очень очень много, их можно и нужно менять, чтобы добиться более хорошего качества.

Рекомендую обратить внимание на некоторые вещи в первую очередь
1. Learning rate
2. Batch size
3. Точность вычислений
4. Scheduler warmup steps
5. Weight Decay
6. Train epochs

In [165]:
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    num_train_epochs=5,
    weight_decay=0.1,
    fp16=False,
    fp16_full_eval=False,
    evaluation_strategy="epoch",
    seed=42,
    save_strategy = "epoch",
    save_total_limit=5,
    logging_strategy="epoch",
    report_to=None,
)


trainer = Trainer(
    model=bert,
    args=training_args,
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Ниже примеры обучения и тестирования нашей модели, которыми удобно пользоваться, чтобы не задумываться о рутинных функциях в обучении

In [166]:
# trainer.train()
base_result = trainer.evaluate(encoded_dataset_test)
print(base_result)

{'eval_loss': 0.4730205833911896, 'eval_model_preparation_time': 0.002, 'eval_accuracy': 0.823394495412844, 'eval_runtime': 7.8787, 'eval_samples_per_second': 110.678, 'eval_steps_per_second': 3.554}


# 1

## 1.1

Ниже дан шаблон, который можно руководствоваться для замены слоев. Это просто пример, который можно редактировать. Как минимум на данном этапе он не сможет поддерживать разные ранги для разных проекций.

In [153]:
class SVDLinear(nn.Module):
    def __init__(self, module, rank): # иницилизируйте класс с необходимыми параметрами
        super(SVDLinear, self).__init__()
        # сделайте сингулярное разложение и запишите матрицы из него в параметры
        # не забывайте что мы хотим хранить их эффективно
        self.rank = rank

        U, S, Vh = torch.linalg.svd(module.weight, full_matrices=False)
        U, S, Vh = U[:, :self.rank], S[:self.rank], Vh[:self.rank, :]
        SVh = torch.diag(S) @ Vh
        self.U, self.SVh = torch.nn.Parameter(data=U), torch.nn.Parameter(data=SVh)
        self.bias = module.bias

    def forward(self, input):
        # напишите форвард который будет эффективно использовать матрицы из разложения
        return (input @ self.SVh.T) @ self.U.T + self.bias

    def extra_repr(self):
      return f"rank={self.rank}"


def apply_svd_weights_to_bert(model, rank, layers_to_compress=None):
  if layers_to_compress is None:
    return model

  for name_layer in layers_to_compress:
    if replace_linear_with_svd(model, name_layer, rank):
      return True
  return False


def replace_linear_with_svd(model, name_layer, rank, cumm_name=""):
  # работает рекурсивно по префиксу name_layer
  for name, module in model.named_children():
    new_cumm_name = cumm_name + "." + name
    if name_layer in new_cumm_name and isinstance(module, nn.Linear):
      new_module = SVDLinear(module, rank)
      setattr(model, name, new_module)
    else:
      if cumm_name == "":
        new_cumm_name = name
      else:
        new_cumm_name = cumm_name + "." + name
      replace_linear_with_svd(module, name_layer, rank, new_cumm_name)

## 1.2

In [154]:
def measure_quality(model):
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=encoded_dataset_train,
      eval_dataset=encoded_dataset_test,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
  )
  return trainer.evaluate(encoded_dataset_test)

In [155]:
# чтобы понимать как писать ключи
bert

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [179]:
quality_results_by_blocks = {}

for block_name in tqdm(['bert.encoder', 'bert.pooler', 'classifier']):
  new_bert = BertForSequenceClassification(config=config)
  new_bert.load_state_dict(state_dict)
  apply_svd_weights_to_bert(new_bert, rank=10, layers_to_compress=[
      block_name
  ])
  quality_results_by_blocks[block_name] = measure_quality(new_bert)


[(k, v['eval_accuracy']) for k, v in  quality_results_by_blocks.items()]

  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:04<00:08,  4.44s/it]

 67%|██████▋   | 2/3 [00:08<00:04,  4.22s/it]

100%|██████████| 3/3 [00:13<00:00,  4.34s/it]


[('bert.encoder', 0.7993119266055045),
 ('bert.pooler', 0.819954128440367),
 ('classifier', 0.823394495412844)]

In [194]:
quality_results_by_type_matrix = {}

for block_name in tqdm(['bert.encoder.layer.0.attention.self.query',
                        'bert.encoder.layer.0.attention.self.key',
                        'bert.encoder.layer.0.attention.self.value']):
  new_bert = BertForSequenceClassification(config=config)
  new_bert.load_state_dict(state_dict)
  apply_svd_weights_to_bert(new_bert, rank=3, layers_to_compress=[
      block_name
  ])
  quality_results_by_type_matrix[block_name] = measure_quality(new_bert)

[(k, v['eval_accuracy']) for k, v in  quality_results_by_type_matrix.items()]

  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:02<00:05,  2.66s/it]

 67%|██████▋   | 2/3 [00:06<00:03,  3.43s/it]

100%|██████████| 3/3 [00:10<00:00,  3.65s/it]


[('bert.encoder.layer.0.attention.self.query', 0.8130733944954128),
 ('bert.encoder.layer.0.attention.self.key', 0.8084862385321101),
 ('bert.encoder.layer.0.attention.self.value', 0.8073394495412844)]

## 1.3

In [ ]:
new_bert = BertForSequenceClassification(config=config)
new_bert.load_state_dict(state_dict)


apply_svd_weights_to_bert(new_bert, rank=5, layers_to_compress=[
    'bert.pooler',
    'classifier'
])
trainer = Trainer(
    model=new_bert,
    args=training_args,
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
finetune_result = trainer.evaluate(encoded_dataset_test)
print(finetune_result)

Epoch,Training Loss,Validation Loss


## 1.4

Вывод:
- разложение матриц енкодера большего всего роняет качество в сравнение с более поздними слоями (пулером и классификатором), что представляется соответствующим реальности, сжатие первичных слоев сети мешает ей извлекать наиболее нужные признаки
- если взять в фокус аттеншен матрицы, то сжатие value большего просаживает качество по сравнеию с query матрицой, что выглядит логично, т.к. эта матрица призвана "назначить" вес наиболее важным токенам
- по обучению не успел дождаться, но предположу, что fintune после svd разложений вырастит качество, чем без дообучения

# Задание 2 (5 Баллов)

Данная секция более экспериментальная и направлена на то, чтобы познакомиться с реализацией тензорных разложений.

Выберите небольшое подмножество слоев из прошлого задания, которые хорошо сжимаются. Реализуйте поддержку тензорных разложений вместо сингулярного (скорее всего придется сделать решейп и разложение). Посмотрите как изменилось качество, какой выигрыш у SVD по количеству параметров, а также насколько степень разложения влияет на качество в данном случае.

На выбор вам предлагается использовать либо Tucker Decomposition либо Tensor Train.

На семинаре мы не успели посмотреть на тензорные разложения, внизу пример кода, который демонстрирует как ими можно пользоваться.

In [ ]:
import tensorly as tl
from tensorly.decomposition import tucker, tensor_train

In [ ]:
tl.set_backend('pytorch')

# создаем случайный тензор
tensor = torch.randn(3, 4, 5)

# Применяем Tucker с рангами 2,2,2 (это ранги Core Tensor)
core, factors = tucker(tensor, rank=[2, 2, 2])


In [ ]:
# Видим что размер Core Tensor такая как мы хотели, а остальные факторы будут растягивать его до изначальной размерности
core.size(), [factor.size() for factor in factors]

(torch.Size([2, 2, 2]),
 [torch.Size([3, 2]), torch.Size([4, 2]), torch.Size([5, 2])])

In [ ]:
# Проводим тензорное разложение с рангом 2
factors = tensor_train(tensor, rank=2).factors

In [ ]:
# мы получили 3 тензора, у которых ранг 2, а другие размерности должны соединяться между собой при перемножении.
# в случае тензорного разложения, мы можем решейпать в более высокие размерности, допустим тензор размерности 5, тогда каждый фактор разложения будет меньше рангом
print("Factor tensors:")
for factor in factors:
    print(factor.size())

Factor tensors:
torch.Size([1, 3, 2])
torch.Size([2, 4, 2])
torch.Size([2, 5, 1])


# Бонус (5 баллов)

Попробуйте дообучить тензорные разложения, посмотрите как зависит качество от параметров, насколько это выгоднее SVD с точки зрения параметров и полученного качества